In [164]:
from __future__ import print_function
import pandas as pd
from random import randrange

In [123]:
header = ["color", "diameter", "label"]
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']
]

pd.DataFrame(training_data, columns=header)

,color,diameter,label
0,Green,3,Apple
1,Yellow,3,Apple
2,Red,1,Grape
3,Red,1,Grape
4,Yellow,3,Lemon


In [124]:
def is_numeric(value):
    return isinstance(value, int) or isinstance(value, float)

In [125]:
## Ask question
class Question:
    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" %(header[self.column], condition, str(self.value))

In [126]:
Question(0, 'red'), Question(1, 2)

(Is color == red?, Is diameter >= 2?)

In [127]:
## Partition a dataset.
def partition(rows, question):
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [128]:
q = Question(1, 2)
true_rows, false_rows = partition(training_data, q)
print("True rows: %s" %true_rows, "\nFalse rows: %s" %false_rows)

True rows: [['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']] 
False rows: [['Red', 1, 'Grape'], ['Red', 1, 'Grape']]


In [129]:
## Count labels for each class
def class_counts(rows):
    counts = {}
    for row in rows:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [220]:
A = class_counts(training_data)
print(A)

{'Grape': 2, 'Lemon': 1, 'Apple': 2}


In [222]:
max(set(A), key=A.get)

'Grape'

In [131]:
## Gini impurity defined from -- https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
## Gini_impurity = 1 - Sum(p**2)
def gini(rows):
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [132]:
gini(training_data)

0.6399999999999999

In [133]:
## Information gain is used to decide which feature to split on at each step in building the tree.
## Information gain = entropy(parent) - entropy(children)

def info_gain(left, right, current_uncertainty):
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - (p * gini(left) + (1 - p) * gini(right))

In [134]:
current_uncertainty = gini(training_data)
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [135]:
## Find best split (question) with largest information gain
def find_best_split(rows):
    best_gain = 0
    best_question = None
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1

    for col in range(n_features):
        values = set([row[col] for row in rows])
        for val in values:
            question = Question(col, val)
            true_rows, false_rows = partition(rows, question)
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue
            gain = info_gain(true_rows, false_rows, current_uncertainty)
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [138]:
best_gain, best_question = find_best_split(training_data)
best_gain, best_question

(0.37333333333333324, Is diameter >= 3?)

In [139]:
## Define leaf and node
class Leaf:
    def __init__(self, rows):
        self.predictions = class_counts(rows)
        
class Decision_Node:
    def __init__(self, question, true_branch, false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [140]:
## Build the tree, split until the information gain = 0
## This is the main part
def build_tree(rows):
    gain, question = find_best_split(rows)
    if gain == 0:
        return Leaf(rows)
    true_rows, false_rows = partition(rows, question)
    true_branch = build_tree(true_rows)
    false_branch = build_tree(false_rows)
    return Decision_Node(question, true_branch, false_branch)

In [141]:
## Print the whole tree
def print_tree(node, spacing=""):
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return
    print (spacing + str(node.question))
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [156]:
my_tree = build_tree(training_data)

In [151]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Lemon': 1, 'Apple': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [157]:
def classify(row, node):
    if isinstance(node, Leaf):
        return node.predictions
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [210]:
classify(['Green', 3, 'Apple'], my_tree)

{'Apple': 1}

In [158]:
def print_leaf(counts):
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [159]:
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon']]

In [160]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Lemon': '50%', 'Apple': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Lemon': '50%', 'Apple': '50%'}


In [244]:
## Bagging
def subsample(dataset, n_sample):
    sample = list()
#    n_sample = round(len(dataset) *1.0 / n_samples)
    print(n_sample)
    while len(sample) < n_sample:
        index = randrange(len(dataset))
        sample.append(dataset[index])
    return sample

def bagging_classify(row, trees):
    c = dict()
    [c.update(classify(row, tree)) for tree in trees]
    return max(c)
#return c

In [245]:
def bagging(train, test, sample_size, n_trees):
    trees = list()
    for i in range(n_trees):
        sample = subsample(train, sample_size)
        tree = build_tree(sample)
        print_tree(tree)
        trees.append(tree)
    classes = [bagging_classify(row, trees) for row in test]
    return classes

In [249]:
bagging(training_data, testing_data, 3, 2)

3
Is diameter >= 3?
--> True:
  Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}
3
Is diameter >= 3?
--> True:
  Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


['Apple', 'Apple', 'Grape', 'Grape', 'Apple']